## Reading and cropping firn data

In [22]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import seaborn as sns
import scipy.io as sio
import os
import rasterio as rio
from rasterio.transform import from_origin, from_bounds, Affine
from shapely.geometry import Point, Polygon, mapping
import geopandas as gpd
from fnmatch import fnmatch

In [23]:
# https://gis.stackexchange.com/questions/446462/crop-raster-with-shapefile
def cropRaster(r, shp, name, path):
    from rasterio.mask import mask
    
    with rio.open(r) as src:
        out_image, out_transform = mask(src, shp, crop = True, filled = True) # setting all pixels outside of the feature zone to zero
        out_meta = src.meta

    out_meta.update({"driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "crs":'EPSG:3031',
    "nodata":-9999,
    "transform": out_transform})

    output_file = f'{path}{name}_cropped.tif'

    with rio.open(output_file, "w", **out_meta) as dest:
        dest.write(out_image)
        dest.close()
        
def writeNpToRaster(array, path, name, trans):
    r = rio.open(path + str(name) + '.tif', # save filepath for save
        'w', # 'write' mode
        driver = 'GTiff', # produces a .tif
        height = array.shape[0], # y len
        width = array.shape[1], # x len
        count = 1, # number of bands
        dtype = array.dtype, # get datatype from input array (float)
        crs = 'EPSG:3031', # polar crs
        transform = trans) # transform to projection of another rio DataReader object
    r.write(array,1)
    r.close()
    
def read_Resample(r, factor):
    with rio.open(r) as src:
        data = src.read(1, out_shape = (src.count,
                                    int(src.height * factor),
                                    int(src.width * factor)
                                    ),
                        resampling = Resampling.bilinear
                       )
        # scale image transform
        transform = src.transform * src.transform.scale(
            (src.width / data.shape[-1]),
            (src.height / data.shape[-2])
        )
    return [data, transform]

First, I will open the .nc file and open the respective datasets

In [24]:
os.chdir('/Users/louie.bell/Cambridge/mphil/Essay (Practical) 3/data/altimetry')
filename ='bb0448974g_2_1.h5'
is_H = h5py.File(filename,'r')

In [25]:
alt = is_H['h_alt'] # get altimetry
firn = is_H['h_firn'] # get firn air
time = is_H['time'] # get time in month indexes
alt_uncert = is_H['uncert_alt'] # get altimetry uncertainty
firn_uncert = is_H['uncert_firn'] # get firn uncertainty
x = is_H['x'] # get x coordinate
y = is_H['y'] # get y coordinate

Next I get the year and season for each data point.

In [26]:
times = list(time[0]) # get all times
years = [int(x) for x in times]
yrFracs = [(x % 1) for x in times]
roundedYr = [round(x,2) for x in yrFracs]
seasons = ['DJF', 'MAM', 'JJA','SON']
hSeasons = []
yearSeasons = []

for i in range(len(roundedYr)):
        if roundedYr[i] == 0.16:
            hSeasons.append(seasons[0])
        elif roundedYr[i] == 0.41:
            hSeasons.append(seasons[1])
        elif roundedYr[i] == 0.66:
            hSeasons.append(seasons[2])
        else:
            hSeasons.append(seasons[3])
for i in range(len(years)):
    yrSeas = str(years[i]) + hSeasons[i]
    yearSeasons.append(yrSeas)

Next I write the firn data to raster and flip the orientation while doing so (it's upside down)

In [27]:
# get the transform
transform = from_origin(np.min(is_H['x']),np.max(is_H['y']), 10000, 10000)

for i in range(len(firn)):
    
    print(f'Doing file {yearSeasons[i]}')
    
    firn_day = firn[i,:,:]/365 # convert to daily mean height change
    
    firn_array = np.flipud(firn_day) # FLIP THE ALT DATA
    
    firn_path  = '/Users/louie.bell/Cambridge/mphil/Essay (Practical) 3/data/altimetry/d_air/' # set output path
    
    firn_name = yearSeasons[i] + '_firn_corr' # set filename
    
    writeNpToRaster(firn_array, firn_path, firn_name, transform) # write to raster
    

Doing file 1992MAM
Doing file 1992JJA
Doing file 1992SON
Doing file 1993DJF
Doing file 1993MAM
Doing file 1993JJA
Doing file 1993SON
Doing file 1994DJF
Doing file 1994MAM
Doing file 1994JJA
Doing file 1994SON
Doing file 1995DJF
Doing file 1995MAM
Doing file 1995JJA
Doing file 1995SON
Doing file 1996DJF
Doing file 1996MAM
Doing file 1996JJA
Doing file 1996SON
Doing file 1997DJF
Doing file 1997MAM
Doing file 1997JJA
Doing file 1997SON
Doing file 1998DJF
Doing file 1998MAM
Doing file 1998JJA
Doing file 1998SON
Doing file 1999DJF
Doing file 1999MAM
Doing file 1999JJA
Doing file 1999SON
Doing file 2000DJF
Doing file 2000MAM
Doing file 2000JJA
Doing file 2000SON
Doing file 2001DJF
Doing file 2001MAM
Doing file 2001JJA
Doing file 2001SON
Doing file 2002DJF
Doing file 2002MAM
Doing file 2002JJA
Doing file 2002SON
Doing file 2003DJF
Doing file 2003MAM
Doing file 2003JJA
Doing file 2003SON
Doing file 2004DJF
Doing file 2004MAM
Doing file 2004JJA
Doing file 2004SON
Doing file 2005DJF
Doing file 2

Now I crop the data to the right extent. First I read in my George VI shapefile:

In [28]:
georgevi = gpd.read_file('/Users/louie.bell/Desktop/mphil/Essay (Practical) 3/data/SCAR_coastline_product/georgeVI_poly.shp')
geoms = []
for i in range(len(georgevi)):
    geom = georgevi.loc[i]['geometry']
    geoms.append(geom)

Then I crop with my cropRaster function.

In [29]:
# get firn directory
firn_dir = os.listdir('/Users/louie.bell/Cambridge/mphil/Essay (Practical) 3/data/altimetry/d_air')
firn_path = '/Users/louie.bell/Cambridge/mphil/Essay (Practical) 3/data/altimetry/d_air/'
firn_files = [file for file in firn_dir if fnmatch(file, '*.tif')]

output_path = '/Users/louie.bell/Cambridge/mphil/Essay (Practical) 3/data/altimetry/d_air/d_air_crop/'

for i in range(len(firn_files)):
    
    print(f'Doing file {firn_files[i]}')
    
    dh = firn_path + firn_files[i] # get file path
    
    name, ext = os.path.splitext(firn_files[i]) # get correct filename
    
    cropRaster(dh, geoms, name, output_path) # crop raster
    print(f'Cropping raster as {name}')

Doing file 2002MAM_firn_corr.tif
Cropping raster as 2002MAM_firn_corr
Doing file 1993MAM_firn_corr.tif
Cropping raster as 1993MAM_firn_corr
Doing file 2001JJA_firn_corr.tif
Cropping raster as 2001JJA_firn_corr
Doing file 2000DJF_firn_corr.tif
Cropping raster as 2000DJF_firn_corr
Doing file 2000SON_firn_corr.tif
Cropping raster as 2000SON_firn_corr
Doing file 2014DJF_firn_corr.tif
Cropping raster as 2014DJF_firn_corr
Doing file 2009MAM_firn_corr.tif
Cropping raster as 2009MAM_firn_corr
Doing file 2014SON_firn_corr.tif
Cropping raster as 2014SON_firn_corr
Doing file 1998MAM_firn_corr.tif
Cropping raster as 1998MAM_firn_corr
Doing file 2015JJA_firn_corr.tif
Cropping raster as 2015JJA_firn_corr
Doing file 2016MAM_firn_corr.tif
Cropping raster as 2016MAM_firn_corr
Doing file 2012MAM_firn_corr.tif
Cropping raster as 2012MAM_firn_corr
Doing file 2011JJA_firn_corr.tif
Cropping raster as 2011JJA_firn_corr
Doing file 2010DJF_firn_corr.tif
Cropping raster as 2010DJF_firn_corr
Doing file 2010SON_f